### This code takes the 'sketchy style svg' from https://fskpf.github.io/ as input and converts the svg paths into polygons (x,y coordinates) which can be plotted in Tableau

In [115]:
#Importing required libraries
import numpy as np
import pandas as pd
from xml.dom import minidom
import os
import svg.path as svg

#Setting the directory to the relevant folder
os.chdir('D:/Git Hub Repos and Code yard/Untitled Folder/')

In [ ]:
#Load and parse SVG file into Python
doc = minidom.parse("Sketchy.svg")

In [ ]:
#Get all the g tags. Note: g tags contain the paths
rects = doc.getElementsByTagName("g")

#Instantiate a list to save the paths in each groups aka 'g tags'
paths=[]

#Loop through all the g tags
for rect in rects:
        if rect.getAttribute("clip-path") != "":                #Paths for each shape is in g tags with 'clip-path' attribute
            paths.append(rect.getElementsByTagName("path")) 

#Above for loop creates 'paths' which is a list of lists. This is because each g tag contain two paths - one for outer boundary and other for sketchy fill lines of each shape

In [ ]:
#Next task is to seperate out the paths of outer boundary and inner sketchy fill into two lists

#Instatntiate two lists for the same
out_paths=[]
in_paths=[]

#Loop through all the paths
for path in paths:
    in_paths.append(path[0].getAttribute('d'))                  #d attribute contains the string that defines the path
    out_paths.append(path[1].getAttribute('d'))

In [ ]:
#Now that we have separated out the d strings, we have to parse them back to path objects inorder to do further manipulations

#Instatiate new lists to save parsed d strings
out_paths_parsed=[]
in_paths_parsed=[]

#loop over each list and parse each d string element
for i in range(0,len(out_path)):
    out_paths_parsed.append(svg.parse_path(out_paths[i]))
    in_paths_parsed.append(svg.parse_path(in_paths[i]))

In [160]:
#Now let sample points from each path separately. First up the outer paths

#Instantiate arrays for X cordinate,Y cordinate, Shape indicator (i.e, first bar, second bar etc in a bar chart), and order for plotting in tableau
out_x=[]
out_y=[]
out_shape=[]
out_order=[]

for i in range(0,len(out_paths_parsed)):                        #looping through all the paths in the array
    for o,j in enumerate(np.arange(0,1,1/4500),1):              #sampling 4500 points from each path
        out_x.append(out_paths_parsed[i].point(j).real)         # extracting x coordinate; which is the real part
        out_y.append(out_paths_parsed[i].point(j).imag)         # extracting y coordinate; which is the imag part
        out_shape.append(i)                                     #Update shape list
        out_order.append(o)                                     #Update order list

#Now lets save the extracted information as a dataframe
Outer_xy = pd.DataFrame({"shape":out_shape,"order":out_order,"x coordinate":out_x,"y coordinate":out_y})

In [161]:
#Next up, lets recreate the above code for inner paths

in_x=[]
in_y=[]
in_shape=[]
in_order=[]
for i in range(0,len(in_paths_parsed)):
    for o,j in enumerate(np.arange(0,1,1/4500),1):              #Using arange because range() only accepts integers 
        in_x.append(in_paths_parsed[i].point(j).real)
        in_y.append(in_paths_parsed[i].point(j).imag)
        in_shape.append(i+20)                                   # Added 20 just to distinguish it from the shapes of out path
        in_order.append(o)
        
#Now lets save this information as a dataframe as well
Inner_xy = pd.DataFrame({"shape":in_shape,"order":in_order,"x coordinate":in_x,"y coordinate":in_y})

In [162]:
#Lets combine both the dataframes and export as a csv

Sketchy_style_backend=Outer_xy.append(Inner_xy)
Sketchy_style_backend.to_csv('Sketchy_style_backend.csv')